In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python

import io # Input/Output Module
import os # OS interfaces
import cv2 # OpenCV package
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library
from PIL import Image

from urllib import request # module for opening HTTP requests
from matplotlib import pyplot as plt # Plotting library
import sklearn
from sklearn import decomposition
from skimage.feature import hog
from scipy.spatial.distance import cosine
from scipy.spatial.distance import cdist
from scipy.stats import mode
import tensorflow as tf
from sklearn.metrics import precision_recall_fscore_support
from sklearn.cluster import KMeans

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.neighbors import NearestNeighbors
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import StratifiedKFold

2024-04-14 21:57:53.704421: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-14 21:57:53.704533: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-14 21:57:53.851787: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


<div style="width:100%; height:140px">
    <img src="https://www.kuleuven.be/internationaal/thinktank/fotos-en-logos/ku-leuven-logo.png/image_preview" width = 300px, heigh = auto align=left>
</div>


KUL H02A5a Computer Vision: Group Assignment 1
---------------------------------------------------------------
Student numbers: <span style="color:red">r0999063, r0967112, r0753150, r0775439, r0967921</span>.

The goal of this assignment is to explore more advanced techniques for constructing features that better describe objects of interest and to perform face recognition using these features. This assignment will be delivered in groups of 5 (either composed by you or randomly assigned by your TA's).

In this assignment you are a group of computer vision experts that have been invited to ECCV 2021 to do a tutorial about  "Feature representations, then and now". To prepare the tutorial you are asked to participate in a kaggle competition and to release a notebook that can be easily studied by the tutorial participants. Your target audience is: (master) students who want to get a first hands-on introduction to the techniques that you apply.

---------------------------------------------------------------
This notebook is structured as follows:
0. Data loading & Preprocessing
1. Feature Representations
2. Evaluation Metrics 
3. Classifiers
4. Experiments
5. Publishing best results
6. Discussion

Make sure that your notebook is **self-contained** and **fully documented**. Walk us through all steps of your code. Treat your notebook as a tutorial for students who need to get a first hands-on introduction to the techniques that you apply. Provide strong arguments for the design choices that you made and what insights you got from your experiments. Make use of the *Group assignment* forum/discussion board on Toledo if you have any questions.

Fill in your student numbers above and get to it! Good luck! 


<div class="alert alert-block alert-info">
<b>NOTE:</b> This notebook is just a example/template, feel free to adjust in any way you please! Just keep things organised and document accordingly!
</div>

<div class="alert alert-block alert-info">
<b>NOTE:</b> Clearly indicate the improvements that you make!!! You can for instance use titles like: <i>3.1. Improvement: Non-linear SVM with RBF Kernel.<i>
</div>
    
---------------------------------------------------------------
# 0. Data loading & Preprocessing

## 0.1. Loading data
The training set is many times smaller than the test set and this might strike you as odd, however, this is close to a real world scenario where your system might be put through daily use! In this session we will try to do the best we can with the data that we've got! 

In [ ]:
# Input data files are available in the read-only "../input/" directory

train = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/train_set.csv', index_col = 0)
train.index = train.index.rename('id')

test = pd.read_csv(
    '/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/test_set.csv', index_col = 0)
test.index = test.index.rename('id')

# read the images as numpy arrays and store in "img" column
train['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/train/train_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in train.iterrows()]

test['img'] = [cv2.cvtColor(np.load('/kaggle/input/kul-h02a5a-computer-vision-ga1-2024/test/test_{}.npy'.format(index), allow_pickle=False), cv2.COLOR_BGR2RGB) 
                for index, row in test.iterrows()]
  

train_size, test_size = len(train),len(test)

"The training set contains {} examples, the test set contains {} examples.".format(train_size, test_size)

*Note: this dataset is a subset of the* [*VGG face dataset*](https://www.robots.ox.ac.uk/~vgg/data/vgg_face/).

## 0.2. A first look
Let's have a look at the data columns and class distribution.

In [ ]:
# The training set contains an identifier, name, image information and class label
train.head(1)

In [ ]:
# The test set only contains an identifier and corresponding image information.

test.head(1)

In [ ]:
# The class distribution in the training set:
train.groupby('name').agg({'img':'count', 'class': 'max'})

Note that **Jesse is assigned the classification label 1**, and **Mila is assigned the classification label 2**. The dataset also contains 20 images of **look alikes (assigned classification label 0)** and the raw images. 

## 0.3. Preprocess data
### 0.3.1 Example: HAAR face detector
In this example we use the [HAAR feature based cascade classifiers](https://opencv-python-tutroals.readthedocs.io/en/latest/py_tutorials/py_objdetect/py_face_detection/py_face_detection.html) to detect faces, then the faces are resized so that they all have the same shape. If there are multiple faces in an image, we only take the first one. 

<div class="alert alert-block alert-info"> <b>NOTE:</b> You can write temporary files to <code>/kaggle/temp/</code> or <code>../../tmp</code>, but they won't be saved outside of the current session
</div>


In [ ]:
class HAARPreprocessor():
    """Preprocessing pipeline built around HAAR feature based cascade classifiers. """
    
    def __init__(self, path, face_size):
        self.face_size = face_size
        file_path = os.path.join(path, "haarcascade_frontalface_default.xml")
        if not os.path.exists(file_path): 
            if not os.path.exists(path):
                os.mkdir(path)
            self.download_model(file_path)
        
        self.classifier = cv2.CascadeClassifier(file_path)
  
    def download_model(self, path):
        url = "https://raw.githubusercontent.com/opencv/opencv/master/data/"\
            "haarcascades/haarcascade_frontalface_default.xml"
        
        with request.urlopen(url) as r, open(path, 'wb') as f:
            f.write(r.read())
            
    def detect_faces(self, img):
        """Detect all faces in an image."""
        
        img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        return self.classifier.detectMultiScale(
            img_gray,
            scaleFactor=1.2,
            minNeighbors=5,
            minSize=(30, 30),
            flags=cv2.CASCADE_SCALE_IMAGE
        )
        
    def extract_faces(self, img):
        """Returns all faces (cropped) in an image."""
        
        faces = self.detect_faces(img)

        return [img[y:y+h, x:x+w] for (x, y, w, h) in faces]
    
#     def preprocess(self, data_row):
#         faces = self.extract_faces(data_row['img'])
        
#         # if no faces were found, return None
#         if len(faces) == 0:
#             nan_img = np.empty(self.face_size + (3,))
#             nan_img[:] = np.nan
#             return nan_img
        
#         # only return the first face
#         return cv2.resize(faces[0], self.face_size, interpolation = cv2.INTER_AREA)
    
    def preprocess(self, image):
        faces = self.extract_faces(image)

        # if no faces were found, return None
        if len(faces) == 0:
            nan_img = np.empty(self.face_size + (3,))
            nan_img[:] = np.nan
            return nan_img

        face_list = [cv2.resize(face, self.face_size, interpolation = cv2.INTER_AREA) for face in faces]
        # only return the second face if multiple faces available in the image
        return list(face_list)
            
#     def __call__(self, data):
#         return np.stack([self.preprocess(row) for _, row in data.iterrows()]).astype(int)

def preprocess_data(data):
    preprocessor = HAARPreprocessor(path='../../tmp', face_size=FACE_SIZE)
    new_data = []
    rows, _ = data.shape
    for r in range(rows):
        # print(r)
        faces = preprocessor.preprocess(data.iloc[r]['img'])
        num_faces = len(faces)
        if num_faces == 0:
            continue  # Skip this row if no faces are found
        for i in range(num_faces):
            new_row = data.iloc[r].copy()
            new_row['img'] = faces[i]
            new_data.append(new_row)
    return pd.DataFrame(new_data)

def remove_rows_with_nan_images(data):
    # Create a mask to identify rows with NaN images
    mask = data['img'].apply(lambda x: np.any(np.isnan(x)))

    # Invert the mask to keep rows where img does not contain NaN
    filtered_data = data[~mask]

    return filtered_data

def return_rows_with_nan_images(data):
    # Create a mask to identify rows with NaN images
    mask = data['img'].apply(lambda x: np.any(np.isnan(x)))

    # Invert the mask to keep rows where img does not contain NaN
    filtered_data = data[mask]

**Visualise**

Let's plot a few examples.

In [ ]:
# parameter to play with 
# FACE_SIZE = (100, 100)
FACE_SIZE = (250, 250)

def plot_image_sequence(data, n, imgs_per_row=7):
    n_rows = 1 + int(n/(imgs_per_row+1))
    n_cols = min(imgs_per_row, n)

    f,ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
    for i in range(n):
        if n == 1:
            ax.imshow(data[i])
        elif n_rows > 1:
            ax[int(i/imgs_per_row),int(i%imgs_per_row)].imshow(data[i])
        else:
            ax[int(i%n)].imshow(data[i])
    plt.show()

    
#preprocessed data 
# preprocessor = HAARPreprocessor(path = '../../tmp', face_size=FACE_SIZE)

train_faces = preprocess_data(train)
train_faces_cleaned = remove_rows_with_nan_images(train_faces)
train_faces_cleaned = train_faces_cleaned.reset_index(drop=False, inplace=False) # for plotting later to show misclassifications
wrong_detections = [5, 18, 26, 28, 33, 35, 36, 37, 38, 39, 51, 54, 56, 58, 60, 62, 65, 70, 73, 76, 78, 85, 88, 89, 98, 99]

# remove the rows with the level_0 value in the list wrong_detections
train_faces_cleaned_mcr = train_faces_cleaned.drop(wrong_detections) # mcr - mis-classifications removed
# replace row 34's name with "Michael_Cera" and Class with 0
train_faces_cleaned_mcr.loc[49, 'name'] = "Michael_Cera"
train_faces_cleaned_mcr.loc[49, 'class'] = 0

# use train_faces_cleaned_mcr as the new train variable

# train_X, train_y = preprocessor(train), train['class'].values
# test_X = preprocessor(test)

train_X, train_y = train_faces_cleaned_mcr['img'].values,  train_faces_cleaned_mcr['class'].values
train_X = np.stack(train_faces_cleaned_mcr['img'], dtype=np.uint8)
train_X = np.array([cv2.resize(x, (100,100), interpolation= cv2.INTER_LINEAR) for x in train_X])

Loading our own version of the test data after fixing it manually.

In [ ]:
def load_and_convert_images_to_numpy(source_folder, target_folder, file_extension='.png'):
    """
    Load images from a folder, convert them to numpy arrays, and save as .npy files in a target folder.

    Args:
        source_folder (str): Path to the folder containing the image files.
        target_folder (str): Path to the folder where .npy files will be saved.
        file_extension (str): The file extension of the images to be loaded.
    """
    # Ensure the target directory exists
    os.makedirs(target_folder, exist_ok=True)

    # List all files in the source folder that match the extension
    image_files = [f for f in os.listdir(source_folder) if f.endswith(file_extension)]
    image_files.sort()

    # Process each file
    for file_name in image_files:
        # Load the image using PIL
        image_path = os.path.join(source_folder, file_name)
        image = Image.open(image_path)
        
        # Convert the image to a numpy array
        image_array = np.array(image)

        # Save the array to a file in the target folder
        np.save(os.path.join(target_folder, file_name[:-len(file_extension)]), image_array)

def load_numpy_images(folder_path, indices):
    """
    Load .npy files from a folder and store them in a list.

    Args:
        folder_path (str): Path to the folder containing .npy files.
        indices (list): A list of indices (as strings) which correspond to the filenames of .npy files to load.

    Returns:
        list: A list of numpy arrays loaded from the specified files.
    """
    images = []
    for index in indices:
        file_path = os.path.join(folder_path, f"{index}.npy")
        image = np.load(file_path, allow_pickle=False)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        images.append(image)
    return images

The modified dataset for testing is available at: https://www.kaggle.com/datasets/csap96/cropped-test/data?select=npy

On the right side of this notebook you can click on "Add Input" and search for the following dataset: "cropped_test_group13_KULCV2324" before running the following code.

In [ ]:
# Define paths
# cropped_folder = '/kaggle/input/cropped-test/cropped'
# numpy_folder = os.path.join(cropped_folder, 'npy')

# # Convert images to numpy arrays and save them
# load_and_convert_images_to_numpy(cropped_folder, numpy_folder)

numpy_folder = '/kaggle/input/cropped-test/npy/npy'
# Load and convert numpy files back to images
indices = [str(n) for n in range(1816)]
test_X_cropped = load_numpy_images(numpy_folder, indices) # this needs to be resized
test_X = np.array([cv2.resize(x, (100,100), interpolation= cv2.INTER_LINEAR) for x in test_X_cropped])

In [ ]:
test_X[0].shape

In [ ]:
# plot faces of Michael and Sarah

plot_image_sequence(train_X[train_y == 0], n=20, imgs_per_row=10)

In [ ]:
# plot faces of Jesse

plot_image_sequence(train_X[train_y == 1], n=30, imgs_per_row=10)

In [ ]:
# plot faces of Mila

#plot_image_sequence(train_X[train_y == 2], n=30, imgs_per_row=10)

## 0.4. Store Preprocessed data (optional)
<div class="alert alert-block alert-info">
<b>NOTE:</b> You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All". Feel free to use this to store intermediary results.
</div>

In [ ]:
# save preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
    
# np.save(os.path.join(prep_path, 'train_X.npy'), train_X)
# np.save(os.path.join(prep_path, 'train_y.npy'), train_y)
# np.save(os.path.join(prep_path, 'test_X.npy'), test_X)

# load preprocessed data
# prep_path = '/kaggle/working/prepped_data/'
# if not os.path.exists(prep_path):
#     os.mkdir(prep_path)
# train_X = np.load(os.path.join(prep_path, 'train_X.npy'))
# train_y = np.load(os.path.join(prep_path, 'train_y.npy'))
# test_X = np.load(os.path.join(prep_path, 'test_X.npy'))

Now we are ready to rock!

# 0.5. Augmentations

Need to update this further.

In [ ]:
def add_watermark(img, watermark_text="Watermark", position=(5,
                                                             33),
                                                             opacity=0.2,
                                                             font_scale=0.5,
                                                             color=(255, 255, 255),
                                                             background_color=(0, 0, 0)):

    # Ensure img is a contiguous array of type uint8
    img = np.ascontiguousarray(img).astype(np.uint8)

    # Prepare overlay
    overlay = img.copy()
    overlay = np.ascontiguousarray(overlay).astype(np.uint8)

    font = cv2.FONT_HERSHEY_SIMPLEX
    # Calculate text size
    (text_width, text_height), _ = cv2.getTextSize(watermark_text, font, font_scale, thickness=2)
    text_x, text_y = position

    # Adjust rectangle position so it serves as a background
    rect_start = (text_x, text_y - text_height - 5)
    rect_end = (text_x + text_width, text_y + 10)

    # Draw the rectangle
    cv2.rectangle(overlay, rect_start, rect_end, background_color, thickness=cv2.FILLED)

    # Draw the text on top
    text_position = (text_x, text_y)
    cv2.putText(overlay, watermark_text, text_position, font, font_scale, color, 2)

    # Blend the overlay
    cv2.addWeighted(overlay, opacity, img, 1 - opacity, 0, img)

    return img

def random_flip_rotate(image):
    """
    Randomly flips and rotates the input image.

    Args:
        image: TensorFlow tensor representing the input image.

    Returns:
        Augmented image with random flips and rotations.
    """
    image = tf.image.random_flip_left_right(image)
    # k = tf.random.uniform(shape=[], minval=0, maxval=4, dtype=tf.int32)
    return image

def color_jitter(image, brightness=0.1, contrast=0.3, saturation=0.2, hue=0.15):
    """
    Randomly adjusts the brightness, contrast, saturation, and hue of an input image in integer format.
    Converts image to float [0, 1], applies adjustments, and converts back to integer [0, 255].

    Args:
        image: TensorFlow tensor representing the input image in integer format [0, 255].
        brightness: Maximum delta for brightness adjustment.
        contrast: Maximum delta for contrast adjustment.
        saturation: Maximum delta for saturation adjustment.
        hue: Maximum delta for hue adjustment.

    Returns:
        Augmented image with color jittering in integer format [0, 255].
    """
    # Convert image to float [0, 1]
    image_float = tf.cast(image, tf.float32) / 255.0

    # Apply color jitter
    image_float = tf.image.random_brightness(image_float, max_delta=brightness)
    image_float = tf.image.random_contrast(image_float, lower=1-contrast, upper=1+contrast)
    image_float = tf.image.random_saturation(image_float, lower=1-saturation, upper=1+saturation)
    image_float = tf.image.random_hue(image_float, max_delta=hue)

    # Clip the image data to ensure it's in the valid range
    image_float = tf.clip_by_value(image_float, 0.0, 1.0)

    # Convert back to integer format [0, 255]
    aug_img = tf.cast(image_float * 255, tf.uint8)

    return aug_img

def add_noise(image, noise_level=0.05):
    """
    Adds random noise to the input image. Assumes image is in [0, 255] integer format.

    Args:
        image: TensorFlow tensor representing the input image in [0, 255] integer format.
        noise_level: Level of noise to be added, scaled for images in [0, 255] format.

    Returns:
        Augmented image with added noise, returned in the same format as the input.
    """
    # Convert image to float [0, 1] for processing
    image_float = tf.cast(image, tf.float32) / 255.0

    # Adjust noise level for [0, 1] scale
    adjusted_noise_level = noise_level * 255.0

    # Generate noise and add to the image
    noise = tf.random.normal(shape=tf.shape(image_float), mean=0.0, stddev=adjusted_noise_level / 255.0)
    noisy_image = tf.clip_by_value(image_float + noise, 0.0, 1.0)

    # Convert back to [0, 255] integer format
    noisy_image = tf.cast(noisy_image * 255, tf.uint8)

    return noisy_image

def random_erasing(image, sl=0.02, sh=0.4, r1=0.3):
    """
    Randomly erases a rectangle region in the input image.

    Args:
        image: TensorFlow tensor representing the input image.
        sl: Minimum proportion of erased area.
        sh: Maximum proportion of erased area.
        r1: Aspect ratio of the erased region.

    Returns:
        Augmented image with random erasing.
    """
    image_shape = tf.shape(image)
    h, w, _ = tf.unstack(image_shape)
    area = tf.cast(w * h, tf.float32)

    target_area = tf.random.uniform([], minval=sl, maxval=sh) * area
    aspect_ratio = tf.random.uniform([], minval=r1, maxval=1/r1)

    h_new = tf.cast(tf.round(tf.sqrt(target_area / aspect_ratio)), tf.int32)
    w_new = tf.cast(tf.round(tf.sqrt(target_area * aspect_ratio)), tf.int32)

    # Ensure the rectangle fits within the image
    h_new = tf.minimum(h, h_new)
    w_new = tf.minimum(w, w_new)

    x1 = tf.random.uniform([], minval=0, maxval=(h - h_new + 1), dtype=tf.int32)
    y1 = tf.random.uniform([], minval=0, maxval=(w - w_new + 1), dtype=tf.int32)

    mask = tf.ones([h, w, 3], dtype=image.dtype)
    erase_block = tf.zeros([h_new, w_new, 3], dtype=image.dtype)

    # Calculate padding dimensions correctly
    padding_top, padding_bottom = x1, (h - x1 - h_new)
    padding_left, padding_right = y1, (w - y1 - w_new)

    erase_block_padded = tf.pad(erase_block, [[padding_top, padding_bottom], [padding_left, padding_right], [0, 0]], constant_values=1)

    # Apply the mask to the image
    image = image * erase_block_padded
    return image


# Function to apply Gaussian blur on an image using OpenCV
def gaussian_blur(image, blur_intensity=5):
    # Apply Gaussian Blur
    blurred_image = cv2.GaussianBlur(image, (blur_intensity, blur_intensity), 0)
    return blurred_image

def sharpen_image(image):
    """
    Sharpens an image using a specific sharpening kernel.
    
    Args:
    image (numpy.ndarray): The input image as a numpy array.
    
    Returns:
    numpy.ndarray: The sharpened image.
    """
    # Define the sharpening kernel, the numbers can be adjusted to change the effect
    sharpening_kernel = np.array([[-1, -1, -1],
                                  [-1,  9, -1],
                                  [-1, -1, -1]])
    
    # Apply the sharpening kernel to the input image
    sharpened_image = cv2.filter2D(image, -1, sharpening_kernel)
    
    return sharpened_image

def pixelate_image(image, pixel_size=3):
    """
    Pixelates an image by reducing its resolution and then resizing it back to its original size.
    
    Args:
    image (numpy.ndarray): The input image as a numpy array.
    pixel_size (int): The size of each block of pixels in the pixelated image.
    
    Returns:
    numpy.ndarray: The pixelated image.
    """
    # Check the input image's dimensions
    height, width = image.shape[:2]
    
    # Resize the image to a smaller size
    small_image = cv2.resize(image, (width // pixel_size, height // pixel_size), interpolation=cv2.INTER_LINEAR)
    
    # Scale it back up to the original size
    pixelated_image = cv2.resize(small_image, (width, height), interpolation=cv2.INTER_NEAREST)
    
    return pixelated_image

def apply_augmentations(train_X, train_y):
    """
    Applies augmentations to the training set.

    Args:
        train_X: Training set features.
        train_y: Training set labels.

    Returns:
        Augmented training set features and labels.
    """
    augmentations = [random_flip_rotate,
                     color_jitter,
                     add_noise,
                     random_erasing,
                     add_watermark,
                     gaussian_blur,
                     sharpen_image,
                     pixelate_image]
    new_train_X = []
    new_train_y = []
    for i in range(len(train_X)):

        image = train_X[i]
        # add the original image
        new_train_X.append(image)
        new_train_y.append(train_y[i])

        for augmentation in augmentations:

            aug_image = augmentation(image)
            # add the augmented image
            new_train_X.append(aug_image)
            new_train_y.append(train_y[i])
    return np.array(new_train_X), np.array(new_train_y)

temp_x = list(train_X)
temp_y = train_y

for x in [65,35,28,24,23,14]:
    temp_x.pop(x)
    temp_y = np.delete(temp_y, x)
temp_x = np.array(temp_x)

new_train_X, new_train_y = apply_augmentations(temp_x, temp_y)
new_train_X.shape, new_train_y.shape

plot_image_sequence(new_train_X, n=20, imgs_per_row=10)

# 1. Feature Representations
## 1.0. Example: Identify feature extractor
Our example feature extractor doesn't actually do anything... It just returns the input:
$$
\forall x : f(x) = x.
$$

It does make for a good placeholder and baseclass ;).

In [ ]:
class IdentityFeatureExtractor:
    """A simple function that returns the input"""

    def transform(self, X):
        return X

    def convertToGray(self,X):
        grayImages = np.empty((np.shape(X)[0],np.shape(X)[1],np.shape(X)[2]))
        for index,image in enumerate(X):
          img_float32 = np.float32(image)
          gray = cv2.cvtColor(img_float32, cv2.COLOR_BGR2GRAY)
          grayImages[index] = gray
        return grayImages

    def rotate(self,X):
      rotatedImages = []
      for image in X:
        print(X)
        rotated = cv2.rotate(image, cv2.ROTATE_90_CLOCKWISE)
        rotatedImages.append(rotated)
      return rotatedImages

    def __call__(self, X):
        return self.transform(X)

## 1.1. Baseline 1: HOG feature extractor/Scale Invariant Feature Transform
...

In [ ]:
class HOGFeatureExtractor(IdentityFeatureExtractor):
    """TODO: this feature extractor is under construction"""
    # documentation to refer to: https://github.com/scikit-image/scikit-image/blob/main/skimage/feature/_hog.py
    def __init__(self, **params):
        self.params = params

    def extract_hog_features(self, X,
                             orientations = 9,
                             ppc = 2,
                             cpb = 1,
                             visualise = True,
                             block_norm='L2-Hys',
                             transform_sqrt=True,
                             feature_vector=True,
                             channel_axis=2):

      feature_vector, hog_image = hog(X,
                                      orientations=orientations,
                                      pixels_per_cell=(ppc, ppc),
                                      cells_per_block=(cpb, cpb),
                                      visualize=visualise,
                                      block_norm=block_norm,
                                      transform_sqrt=transform_sqrt,
                                      feature_vector=feature_vector,
                                      channel_axis=channel_axis)
      return feature_vector, hog_image

    def transform(self, X):
        features, hogimage = self.extract_hog_features(X)
        return features, hogimage 


In [ ]:
hogfe = HOGFeatureExtractor()
feature_vector, image = hogfe.transform(new_train_X[0])
plt.imshow(image)

**SIFT**

Scale Invariant Feature Transform is a robust means of identifying areas of interest from an image using Difference of Gaussian blurs and then assigning a signature to those areas of interest with a local gradient of histograms. It is commonly used for performing stitching and correspondance between two images, or for object detection in images which could be incorporated into a query and retrieval system.

Below is the implementation for extracting and describing key features of an image, as well as performing nearest matches between two images and clustering continuous features into a Bag of Words discrete histogram.

In [ ]:
class SIFT_FeatureExtractor(IdentityFeatureExtractor):

  def __init__(self,**params):
    self.params = params
    self.words = None

  # This transform method should return a list of feature descriptors from the entire dataset X
  def transform(self,X):
    all_descriptors = []
    all_keypoints = []
    sift = cv2.SIFT_create(nOctaveLayers = self.params["Octaves"],
                          nfeatures =  self.params["NumberOfFeatures"],
                          contrastThreshold = self.params["ConstrastThreshold"],
                          edgeThreshold = self.params["EdgeThreshold"],
                          sigma = self.params["Smoothing"] )
    for img in X:
        gray = cv2.cvtColor(img.astype(dtype=np.uint8), cv2.COLOR_BGR2GRAY)
        kp, des = sift.detectAndCompute(gray, None)
        all_descriptors.append(des)
        all_keypoints.append(kp)
    return all_keypoints, all_descriptors

  # This method returns the keypoints and corresponding descriptors for ONE image
  def transformOne(self,X,index):
    sift = cv2.SIFT_create(nfeatures =  self.params["NumberOfFeatures"],
                          contrastThreshold = self.params["ConstrastThreshold"],
                          edgeThreshold = self.params["EdgeThreshold"],
                          sigma = self.params["Smoothing"] )
    gray = cv2.cvtColor(X[index].astype(dtype=np.uint8), cv2.COLOR_BGR2GRAY)
    kp, des = sift.detectAndCompute(gray, None)
    return kp, des

  # This method finds and plots points of interest on ONE image
  def showFeatures(self,X,index):
    kp, des = self.transformOne(X,index)
    output_image = cv2.drawKeypoints(X[index], kp, 0, (0, 255, 0),
                                 flags=cv2.DRAW_MATCHES_FLAGS_DEFAULT)
    print("{} features detected".format(np.shape(des)[0]))
    plt.imshow(output_image)
    plt.show()

  # This method finds and plots matches between two images in the dataset
  def match(self,X,index1,index2):
    kp1,des1 = self.transformOne(X,index1)
    kp2,des2 = self.transformOne(X,index2)
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck = True)
    matches = bf.match(des1,des2)
    matches = sorted(matches, key = lambda x:x.distance)

    img_matches = cv2.drawMatches(X[index1], kp1, X[index2], kp2, matches, outImg = np.empty((1,1)))
    plt.imshow(img_matches)
    plt.show()

  # This method gathers descriptors of different lengths into a single array
  def _gather_descriptors(self,descriptors_list):
    descriptors = np.concatenate(descriptors_list, axis=0)
    return descriptors

  # This method initialises a set of K clusters representing different words
  def createKWords(self,descriptors,k):
    descriptors = self._gather_descriptors(descriptors)
    kmeans = KMeans(n_clusters=k,random_state=42)
    kmeans.fit(descriptors)
    self.words = kmeans
    print("{} words initialised from fed descriptors".format(k))

  # This method returns a set of words from a set of descriptors
  def getWords(self,descriptors):
    X_words = []
    for des in descriptors:
      features = np.array([0] * np.shape(self.words.cluster_centers_)[0])
      distance = cdist(des, self.words.cluster_centers_, metric='euclidean')
      argmin = np.argmin(distance, axis = 1)
      for j in argmin:
        features[j] += 1
      X_words.append(features)
    return X_words

  def __call__(self, X):
    return self.transform(X)

With SIFT, we can extract key features and describe them with a histogram of gradients. Furthermore, we can perform matching of features between images for tasks such as stitching and correspondance:

In [ ]:
# Parameters chosen from:
# https://www.researchgate.net/publication/366445603_A_hybrid_approach_for_face_recognition_using_a_convolutional_neural_network_combined_with_feature_extraction_techniques

PARAMS = {
    "Octaves":3,
    "NumberOfFeatures": 80,
    "ConstrastThreshold": 0.04,
    "EdgeThreshold": 10,
    "Smoothing": 1.6,
}

trans = SIFT_FeatureExtractor(**PARAMS)
all_keypoints, all_features = trans(new_train_X) #Note that these are LISTS with a length equal to the training set size
trans.showFeatures(new_train_X,0)
trans.match(new_train_X,0,5)

To convert a set of continuous descriptors into discrete features, we can use Bag of Words representation. This will cluster the descriptors into a set of distinct features which describe the image. This signature is invariant to spatial information.

In [ ]:
numClusters = 20

trans.createKWords(all_features,numClusters)
all_words = trans.getWords(all_features)

plt.figure(figsize=(8, 6))
plt.bar(range(len(all_words[0])),all_words[0])
plt.title('Bag of Words descriptor')
plt.xlabel('Word')
plt.ylabel('Frequency')
plt.show()

We can compare how similar two Bag of Words descriptors are via cosine similarity, which compares relative distribution of words:

In [ ]:
def compare_word_descriptors(words1,words2):
  return 1-cosine(words1,words2)

def test_cosine(stacked_vectors,train_y,test_index):
  cosine_similarities = []
  top_k = int(6)
  for sample in range(np.shape(stacked_vectors)[0]):
    cosine_similarity = 1 - cosine(stacked_vectors[test_index], stacked_vectors[sample])
    cosine_similarities.append(cosine_similarity)
  cosine_similarities = np.array(cosine_similarities)
  ind = np.argpartition(cosine_similarities, -1*top_k)[-1*top_k:]
  return train_y[test_index], mode(train_y[ind[:-1]]).mode, train_y[test_index] == mode(train_y[ind[:-1]]).mode

In [ ]:
index_1 = 0
index_2 = 3
index_3 = 7
plt.imshow(new_train_X[index_1])
plt.show()
plt.imshow(new_train_X[index_2])
plt.show()
plt.imshow(new_train_X[index_3])
plt.show()

cosine_similarity1 = compare_word_descriptors(all_words[index_1],all_words[index_2])
cosine_similarity2 = compare_word_descriptors(all_words[index_1],all_words[index_3])


print("Similarity between visual description of images {} and {} = {}".format(index_1,index_2,cosine_similarity1))
print("Similarity between visual description of images {} and {} = {}".format(index_1,index_3,cosine_similarity2))

We can even determine how discriminative our choice of hyperparameters is on the training set by performing a nearest neighbour search on the word cluster centres for each training image. From this we can calculate accuracies and an F1 score:

In [ ]:
NA_result = []
J_result = []
M_result = []

targets = []
preds = []
for i in range(np.shape(all_words)[0]):
  target, pred, result = test_cosine(all_words,new_train_y,i)
  targets.append(target)
  preds.append(pred)
  if target == 0:
    NA_result.append(result)
  elif target == 1:
    J_result.append(result)
  elif target == 2:
    M_result.append(result)
total_result = NA_result+J_result+M_result
# Calculate precision, recall, and F1-score for each class
precision, recall, f1_score, _ = precision_recall_fscore_support(np.array(targets), np.array(preds), average=None)
# Calculate the average F1-score across all classes
average_f1_score = sum(f1_score) / len(f1_score)
print("Total accuracy = ", total_result.count(True) / (total_result.count(True)+total_result.count(False)))
print("NA accuracy = ", NA_result.count(True) / (NA_result.count(True)+NA_result.count(False)))
print("J accuracy = ", J_result.count(True) / (J_result.count(True)+J_result.count(False)))
print("M accuracy = ", M_result.count(True) / (M_result.count(True)+M_result.count(False)))
print("F1 score = ", average_f1_score)

Furthermore, we can perform a hyperparameter gridsearch to determine a best set of parameters.

In [ ]:
results_dict = {}

for octave in np.arange(3,6,1):
  for contrast in np.arange(0.01,0.07,0.02):
    for edge in np.arange(10,30,10):
      for smoothing in np.arange(1.0,4.0,1.0):
        for k in np.arange(20,50,10):

            PARAMS = {
                    "NumberOfFeatures": 100,
                    "Octaves": octave,
                    "ConstrastThreshold": contrast,
                    "EdgeThreshold": edge,
                    "Smoothing": smoothing,
                      }

            trans = SIFT_FeatureExtractor(**PARAMS)
            all_keypoints, all_features = trans(train_X)
            trans.showFeatures(train_X,0)
            trans.createKWords(all_features,k)
            all_words = trans.getWords(all_features)

            NA_result = []
            J_result = []
            M_result = []
            targets = []
            preds = []
            for i in range(np.shape(all_words)[0]):
              target, pred, result = test_cosine(all_words,train_y,i)
              targets.append(target)
              preds.append(pred)
              if target == 0:
                NA_result.append(result)
              elif target == 1:
                J_result.append(result)
              elif target == 2:
                M_result.append(result)
            total_result = NA_result+J_result+M_result
            # Calculate precision, recall, and F1-score for each class
            precision, recall, f1_score, _ = precision_recall_fscore_support(np.array(targets), np.array(preds), average=None)
            # Calculate the average F1-score across all classes
            average_f1_score = sum(f1_score) / len(f1_score)
            print("Octaves = ", octave, " ContrastThreshold = ", contrast, " EdgeThreshold = ", edge, " Smoothing = ", smoothing, " K = ", k)
            print("F1 score = ", average_f1_score)

            results_dict[(octave,contrast,edge,smoothing,k)] = average_f1_score

In [ ]:
highest_keys = sorted(results_dict, key=lambda x: results_dict[x], reverse=True)[:10]

print("Keys of the 5 highest float values:", highest_keys)

### 1.1.1. t-SNE Plots

T-distributed stochastic neighbour embedding is a means of projecting data in a high dimensional space to a lower space. This improves visualisation of clusters and relations between observations. We can perform this on the K-dimensional visual words descriptors to indicate a level of seperability between the classes:

In [ ]:
from sklearn.manifold import TSNE

PARAMS = {
    "Octaves":4,
    "NumberOfFeatures": 100,
    "ConstrastThreshold": 0.01,
    "EdgeThreshold": 30,
    "Smoothing": 2.0,
}

trans = SIFT_FeatureExtractor(**PARAMS)
all_keypoints, all_features = trans(train_X) #Note that these are LISTS with a length equal to the training set size
numClusters = 70
trans.createKWords(all_features,numClusters)
all_words = trans.getWords(all_features)

X_embedded = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(np.array(all_words))

color_map = {
    0: 'r',   # Red
    1: 'g',   # Green
    2: 'b',   # Blue
}

color_array = []
for i in range(train_y.shape[0]):
      color_array.append(color_map[train_y[i]])

print("Jesse: Green, Mila: Blue, Michael or Sarah: Red")

plt.figure()
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=color_array)
plt.show()

Given the sparsity of the different classes, it is hard to make out distinctive clusters. Hence SIFT features from the unaltered dataset might not be a good medium for performing classification.

Using data augmentation to enrich the training set could be an avenue for improving the discriminator:

In [ ]:
PARAMS = {
    "Octaves":4,
    "NumberOfFeatures": 100,
    "ConstrastThreshold": 0.01,
    "EdgeThreshold": 30,
    "Smoothing": 2.0,
}

trans = SIFT_FeatureExtractor(**PARAMS)
all_keypoints, all_features = trans(new_train_X) #Note that these are LISTS with a length equal to the training set size
numClusters = 70
trans.createKWords(all_features,numClusters)
all_words = trans.getWords(all_features)

X_embedded = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(np.array(all_words))

color_map = {
    0: 'r',   # Red
    1: 'g',   # Green
    2: 'b',   # Blue
}

color_array = []
for i in range(new_train_y.shape[0]):
      color_array.append(color_map[new_train_y[i]])

print("Jesse: Green, Mila: Blue, Michael or Sarah: Red")

plt.figure()
plt.scatter(X_embedded[:,0],X_embedded[:,1],c=color_array)
plt.show()

From the t-SNE plot of the augmented dataset, we can see that barring a handful of outliers the blue and green dots are fairly seperable.
This suggests that SIFT features enables some good discrimnation between the pictures of Mila Kunis and Jesse Eisenberg.
However, the random positions of the red dots suggest that the added images of Michael Cera and Sarah Nyland confuses the classification. This can be particularly seen by how using a nearest neighbour method via cosine similarity will often misclassify a picture of Mila Kunis as Sarah Nyland.

### 1.1.2. Discussion

Using SIFT, we have demonstrated that an image can be characterised by its local feature descriptors. This representation is far more robust and information enriched than global characterisation methods for matching such as template matching using cross-correlation between images.

The challenge with using SIFT is converting extracted histograms of gradients into a reliable feature space to be used by a classifier. This is addressed by employing K-means clustering to form a Bag of Visual words representation.

Another challenge that we have demonstrated is the choice of hyperparameters for blob detection to find key areas of interest. For different tasks, details at different scales are much more informative than others. For example, discriminating for a more simple task such as between general images of men and women is more dependent on large details whereas for discrimnation between similar looking men and women requires a focus on smaller details.

For the case of discriminating between Jesse Eisenberg and Mila Kunis, SIFT features are shown to be a good choice for a discriminative features space.
However, with the inclusion of similar looking people the task becomes much harder since features across different scales become much more important.

## 1.2. Baseline 2: PCA feature extractor
...

In [ ]:
class PCA_AllColourscales_FeatureExtractor(IdentityFeatureExtractor):
    def __init__(self, train, labels, n_components, size, colour):
      self.n_components = n_components
      self.size = size
      self.train = train
      self.labels = labels
      self.colour=colour
      self.dim_reduction()


    # Perform dimensionality reduction
    def dim_reduction(self):
      # If working in greyscale...
      if self.colour == False:
        gray_imgs = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        for img in range(self.train.shape[0]):
          gray_imgs[img,:,:] = cv2.cvtColor(np.squeeze(self.train[img,:,:,:]).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        self.gray_eigenfaces, self.gray_features, self.gray_mean_vec, self.gray_eigenvalues = self.rest_reduct(gray_imgs)
      
      # If working in colourscale...
      elif self.colour == True:
        blue_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        red_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        green_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        for img in range(self.train.shape[0]):
          # Need to split the images into separate colour channels
          B, G, R = cv2.split(self.train[img,:,:,:])
          blue_img[img, :, :] = B
          red_img[img, :, :] = R
          green_img[img, :, :] = G

        self.blue_eigenfaces, self.blue_features, self.blue_mean_vec, self.blue_eigenvalues = self.rest_reduct(blue_img)
        self.red_eigenfaces, self.red_features, self.red_mean_vec, self.red_eigenvalues = self.rest_reduct(red_img)
        self.green_eigenfaces, self.green_features, self.green_mean_vec, self.green_eigenvalues = self.rest_reduct(green_img)
      return self


    # More of the dimensionality reduction method
    def rest_reduct(self, imgstack):
      vec = imgstack.reshape((self.train.shape[0],-1))
      # Obtain vector of mean values for data normalization / centering
      mean_vec = np.mean(vec, 0)
      vec_mean_subtracted = vec - mean_vec
      # Obtain covariance matrix of centred values
      cov_matrix = np.dot(vec_mean_subtracted, vec_mean_subtracted.T)/self.train.shape[0]
      eigenvalues, eigenvectors = np.linalg.eig(cov_matrix)
      # Sort eigenvectors by ascending eigenvalues
      sorted_indices = np.argsort(eigenvalues)[::-1]
      eigenvalues = eigenvalues[sorted_indices]
      eigenvectors = eigenvectors[:, sorted_indices]
      sum = np.sum(eigenvalues)
      sum_95 = self.n_components/100 * sum
      temp=0
      p=0
      while temp < sum_95:
        temp += eigenvalues[p]
        p += 1
      eigenfaces = np.dot(vec_mean_subtracted.T, eigenvectors)
      eigenfaces= eigenfaces / np.linalg.norm(eigenfaces, axis=0)
      eigenfaces = eigenfaces[:,:p]
      features = np.dot(vec_mean_subtracted, eigenfaces)
      return eigenfaces, features, mean_vec, eigenvalues

    # This method controls getting the appropriate eigenfaces according to the colourscale being used
    def get_eigenfaces(self):
      if self.colour == False:
        return self.gray_eigenfaces
      else:
        return self.red_eigenfaces, self.green_eigenfaces, self.blue_eigenfaces

    # This method controls getting the appropriate features according to the colourscale being used
    def get_features(self):
      if self.colour == False:
        return self.gray_features
      else:
        return self.red_features, self.green_features, self.blue_features
    
    # Transform input data to existing eigenvectors
    def transform(self,X):
      #if grey
      if self.colour == False:
        gray_img = np.zeros((X.shape[0], X.shape[1], X.shape[2]), dtype=np.int32)
        for img in range(X.shape[0]):
            gray_img[img,:,:] = cv2.cvtColor(np.squeeze(X[img,:,:,:]).astype(np.uint8), cv2.COLOR_RGB2GRAY)
        gray_vec = gray_img.reshape((X.shape[0],-1))
        vec_mean_subtracted = gray_vec - self.gray_mean_vec
        features = np.dot(vec_mean_subtracted, self.eigenfaces)
        return features
      #if colour
      elif self.colour == True:
        blue_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        red_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)
        green_img = np.zeros((self.train.shape[0], self.train.shape[1], self.train.shape[2]), dtype=np.int32)

        for img in range(X.shape[0]):
          B, G, R = cv2.split(self.X[img,:,:,:])
          blue_img[img, :, :] = B
          red_img[img, :, :] = R
          green_img[img, :, :] = G
        red_vec = red_img.reshape((X.shape[0], -1))
        Rvec_mean_subtracted = red_vec - self.red_mean_vec
        Rfeatures = np.dot(Rvec_mean_subtracted, self.red_eigenfaces)

        blue_vec = blue_img.reshape((X.shape[0], -1))
        Bvec_mean_subtracted = blue_vec - self.blue_mean_vec
        Bfeatures = np.dot(Bvec_mean_subtracted, self.blue_eigenfaces)

        green_vec = green_img.reshape((X.shape[0], -1))
        Gvec_mean_subtracted = green_vec - self.green_mean_vec
        Gfeatures = np.dot(Gvec_mean_subtracted, self.green_eigenfaces)
        return Rfeatures, Gfeatures, Bfeatures

    def inverse_transform(self, X, n_eigenfaces_vec):
      if self.colour == False:
        if n_eigenfaces_vec == 0:
            gray_img =  self.gray_mean_vec.reshape(X.shape[0], self.size[0], self.size[1])
            return gray_img
        else:
          rec_img_vec = np.dot(X[:,:n_eigenfaces_vec], self.gray_eigenfaces.T[:n_eigenfaces_vec,:])
          rec_img_vec += self.gray_mean_vec
          rec_gray_img = rec_img_vec.reshape(X.shape[0], self.size[0], self.size[1])
          return rec_gray_img
      else:
        reconstructed_red = self.inv_method(X[0], n_eigenfaces_vec[0], 'red')
        reconstructed_green = self.inv_method(X[1], n_eigenfaces_vec[1], 'green')
        reconstructed_blue = self.inv_method(X[2], n_eigenfaces_vec[2], 'blue')
        if n_eigenfaces_vec[0] == 0:
          return cv2.merge([reconstructed_blue, reconstructed_green, reconstructed_red])
        else:
          total_images = np.zeros((reconstructed_red.shape[0], reconstructed_red.shape[1], reconstructed_red.shape[2], 3), dtype=np.int32)
          for i in range(reconstructed_red.shape[0]):
            total_images[i, :, :] = cv2.merge([reconstructed_blue[i], reconstructed_green[i], reconstructed_red[i]])
          return total_images


    def inv_method(self, X, n_eigenfaces, colour):
      if n_eigenfaces == 0:
        if colour == 'gray' or colour == 'grey':
          reconstructed =  self.gray_mean_vec.reshape(self.size[0], self.size[1])
        elif colour == 'red':
          reconstructed =  self.red_mean_vec.reshape(self.size[0], self.size[1])
        elif colour == 'green':
          reconstructed =  self.green_mean_vec.reshape(self.size[0], self.size[1])
        elif colour == 'blue':
          reconstructed =  self.blue_mean_vec.reshape(self.size[0], self.size[1])
        return reconstructed
      else:
        if colour == 'gray' or colour == 'grey':
          rec_img_vec = np.dot(X[:,:n_eigenfaces], self.gray_eigenfaces.T[:n_eigenfaces,:])
          rec_img_vec += self.gray_mean_vec
        elif colour == 'red':
          rec_img_vec = np.dot(X[:,:n_eigenfaces], self.red_eigenfaces.T[:n_eigenfaces,:])
          rec_img_vec += self.red_mean_vec
        elif colour == 'green':
          rec_img_vec = np.dot(X[:,:n_eigenfaces], self.green_eigenfaces.T[:n_eigenfaces,:])
          rec_img_vec += self.green_mean_vec
        elif colour == 'blue':
          rec_img_vec = np.dot(X[:,:n_eigenfaces], self.blue_eigenfaces.T[:n_eigenfaces,:])
          rec_img_vec += self.blue_mean_vec

        reconstructed = rec_img_vec.reshape(X.shape[0], self.size[0], self.size[1])
        return reconstructed


    def plot_eigenfaces(self, img_per_row=10):
      if self.colour == False:
        self.plot_eigenfaces_gray(img_per_row)
      elif self.colour == True:
        self.plot_eigenfaces_colour(img_per_row)

    def plot_eigenfaces_gray(self,img_per_row = 10):
      n_rows = 1+ int(self.gray_eigenfaces.shape[1]/(img_per_row))
      n_cols = min(img_per_row, self.gray_eigenfaces.shape[1])

      f, ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))

      for i ,ax in enumerate(ax.flat):
        if i < self.gray_eigenfaces.shape[1]:
          ax.imshow(self.gray_eigenfaces[:,i].reshape(self.size[0], self.size[1]))
        
    def plot_eigenvalues(self):
      if self.colour == False:
        plt.plot(self.gray_eigenvalues)
      else:
        f, ax = plt.subplots(1,3, figsize=(10*1,10*3))
        ax[0].plot(self.blue_eigenvalues)
        ax[1].plot(self.green_eigenvalues)
        ax[2].plot(self.red_eigenvalues)

    def plot_eigenfaces_colour(self, img_per_row=10):
      n_cols = img_per_row
      n_rowsR = 1+ int(self.red_eigenfaces.shape[1]/(img_per_row))
      n_rowsG = 1+ int(self.green_eigenfaces.shape[1]/(img_per_row))
      n_rowsB = 1+ int(self.blue_eigenfaces.shape[1]/(img_per_row))

      fig1, ax1 = plt.subplots(n_rowsR,n_cols, figsize=(10*n_cols,10*n_rowsR))
      for i, ax1 in enumerate(ax1.flat):
        if i < self.red_eigenfaces.shape[1]:
          ax1.imshow(self.red_eigenfaces[:, i].reshape(self.size[0], self.size[1]))

      fig2, ax2 = plt.subplots(n_rowsG,n_cols, figsize=(10*n_cols,10*n_rowsG))
      for i, ax2 in enumerate(ax2.flat):
        if i < self.green_eigenfaces.shape[1]:
          ax2.imshow(self.green_eigenfaces[:, i].reshape(self.size[0], self.size[1]))

      fig3, ax3 = plt.subplots(n_rowsB,n_cols, figsize=(10*n_cols,10*n_rowsB))
      for i, ax3 in enumerate(ax3.flat):
        if i < self.blue_eigenfaces.shape[1]:
          ax3.imshow(self.blue_eigenfaces[:, i].reshape(self.size[0], self.size[1]))

    def plot_feature_space_gray(self):
      plt.scatter(self.gray_features[np.where(self.labels == 0)[0], 0], self.gray_features[np.where(self.labels==0)[0], 1], c="orange", label="MichaelCera")
      plt.scatter(self.gray_features[np.where(self.labels == 1)[0], 0], self.gray_features[np.where(self.labels==1)[0], 1], c="red", label="JesseEisenberg")
      plt.scatter(self.gray_features[np.where(self.labels == 2)[0], 0], self.gray_features[np.where(self.labels==2)[0], 1], c="blue", label="MilaKunis")
      plt.title("Plotted PCA of Grey Images")
      plt.legend()
      plt.xlabel('PC1')
      plt.ylabel('PC2')
      plt.legend()
      plt.show()

    def plot_feature_space_colour(self):
      fig, ax = plt.subplots(1,3)

      ax[0].scatter(self.blue_features[np.where(self.labels == 0)[0], 0], self.blue_features[np.where(self.labels == 0)[0], 1], c="orange", label="MichaelCera")
      ax[0].scatter(self.blue_features[np.where(self.labels == 1)[0], 0], self.blue_features[np.where(self.labels == 1)[0], 1], c="red", label="JesseEisenberg")
      ax[0].scatter(self.blue_features[np.where(self.labels == 2)[0], 0], self.blue_features[np.where(self.labels == 2)[0], 1], c="blue", label="MilaKunis")
      ax[0].set_title("B Channel")
      ax[0].set_xlabel("PC1")
      ax[0].set_ylabel("PC2")

      ax[1].scatter(self.red_features[np.where(self.labels == 0)[0], 0], self.red_features[np.where(self.labels == 0)[0], 1], c="orange", label="MichaelCera")
      ax[1].scatter(self.red_features[np.where(self.labels == 1)[0], 0], self.red_features[np.where(self.labels == 1)[0], 1], c="red", label="JesseEisenberg")
      ax[1].scatter(self.red_features[np.where(self.labels == 2)[0], 0], self.red_features[np.where(self.labels == 2)[0], 1], c="blue", label="MilaKunis")
      ax[1].set_title("R Channel")
      ax[1].set_xlabel("PC1")
      ax[1].set_ylabel("PC2")

      ax[2].scatter(self.green_features[np.where(self.labels == 0)[0], 0], self.green_features[np.where(self.labels == 0)[0], 1], c="orange", label="MichaelCera")
      ax[2].scatter(self.green_features[np.where(self.labels == 1)[0], 0], self.green_features[np.where(self.labels == 1)[0], 1], c="red", label="JesseEisenberg")
      ax[2].scatter(self.green_features[np.where(self.labels == 2)[0], 0], self.green_features[np.where(self.labels == 2)[0], 1], c="blue", label="MilaKunis")
      ax[2].set_title("G Channel")
      ax[2].set_xlabel("PC1")
      ax[2].set_ylabel("PC2")

      # The following legend applies to all three subplots.
      ax[2].legend(bbox_to_anchor=(1.01, 1), loc='upper left')
      plt.tight_layout()
      plt.show()

### 1.2.1. Eigenface Plots
Let us run PCA analyses to examine the eigenfaces of the images, both on colourscale and on greyscale.

In [ ]:
# Let's first run colourscale PCAs and plot the eigenfaces.
facesize=(100, 100)
pca_colour = PCA_AllColourscales_FeatureExtractor(train_X, train_y, 95, facesize, True)
features_red, features_green, features_blue = pca_colour.get_features()
eigenfaces_red, eigenfaces_blue, eigenfaces_green = pca_colour.get_eigenfaces()
pca_colour.plot_eigenfaces(10)

In [ ]:
# Now taking a look at the eigenvalues of the model to examine variance in the components
pca_colour.plot_eigenvalues()

In [ ]:
# Let's look at the greyscale version now
pca_grey = PCA_AllColourscales_FeatureExtractor(train_X, train_y, 95, facesize, False)
features_grey = pca_grey.get_features()
eigenfaces_grey = pca_grey.get_eigenfaces()
print(eigenfaces_grey.shape)
pca_grey.plot_eigenfaces(10)

In [ ]:
# Same as before, but greyscale
pca_grey.plot_eigenvalues()

### 1.2.2. Feature Space Plots
...

In [ ]:
# Now, let's plot the feature space of the RGB images
pca_colour.plot_feature_space_colour()

In [ ]:
# Same thing, but for greyscale
pca_grey.plot_feature_space_gray()

In [ ]:
# What if we try to rebuild the images?
pca_colour = PCA_AllColourscales_FeatureExtractor(train_X, train_y, 95, facesize, True)
features_red, features_green, features_blue = pca_colour.get_features()
n_features = features_red.shape[1]
img_per_row = 10
n_rows = 1+ int(n_features/(img_per_row))
n_cols = min(img_per_row, n_features)
f, ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
for i in range(n_features+1):
    rec_img = pca_colour.inverse_transform([features_red, features_green, features_blue], [i, i, i])
    ax[int(i/img_per_row),int(i%img_per_row)].imshow(np.squeeze(rec_img[0]), cmap='gray')

In [ ]:
# Same thing, but now for greyscale
pca_grey = PCA_AllColourscales_FeatureExtractor(train_X, train_y, 95, facesize, False)
features = pca_grey.get_features()
feature_img = features[0,:]
n_features = features.shape[1]
img_per_row = 10
n_rows = 1+ int(features.shape[1]/(img_per_row))
n_cols = min(img_per_row, features.shape[1])
f, ax = plt.subplots(n_rows,n_cols, figsize=(10*n_cols,10*n_rows))
for i in range(n_features+1):
    rec_img = pca_grey.inverse_transform(feature_img[np.newaxis,:],i)
    ax[int(i/img_per_row),int(i%img_per_row)].imshow(np.squeeze(rec_img), cmap='gray')

### 1.2.3. Discussion
In this section, the principal components for the training images (both in colourscale and in greyscale) were found by converting the images to 2D arrays and performing PCA.

The original images were 3D as they contained stacked colour channel images. Thus, to obtain greyscale images, the opencv package was used to scale the pixel intensities so that the pixel values only ranged between white and black, which can be contained in a single value. When performing PCA on coloured images, the principal components were actually found on the three separate colour channels. The separated colour channels were obtained, again, through using opencv, this time to split the images into BGR channels. In this manner, 2D matrices representing the original images were obtained for PCA.

In preparing the image data for PCA, all the images underwent mean subtraction as a form of data normalization. This normalization step is important in ensuring that features with the maximum variance are created. If the data is not normalized, then the mathematical process may favor features that appear to maximize variance but, in relation to the mean, do not provide as informative components.

The primary components capturing the most variation in the data are termed eigenfaces. These eigenfaces arise from determining the eigenvectors of the covariance matrix derived from the 2D matrix. However, in the case of an image size of 100x100, the resulting covariance matrix is massive, at 10000x10000, yet due to the constraints of training data, only 79 eigenvectors possess non-zero eigenvalues. Fortunately, we can exploit the structure of the data matrix to reduce computational complexity and isolate the non-zero eigenvectors. This involves first computing the covariance matrix of the transposed data matrix, now reduced to 80x80. Subsequently, we compute the eigenvectors and eigenvalues of this covariance matrix. Multiplying these eigenvectors with the original 2D matrix yields the eigenfaces of the training data. The eigenvalues provide insights into which eigenfaces capture the most variance within our training dataset.

The optimal value of the number of principal components must retain a percentage of the information that is representative and at the same time significantly reduces the dimensionality of the problem. Therefore, a trade-off between information and dimensionality has to be found.

Examining the eigenvalues derived from PCA exposes the variance encapsulated by individual components. Notably, in our dataset, the initial 25 eigenfaces account for 90% of the total variance, indicating their significant information content. Subsequent eigenfaces contribute progressively less to the overall retained information. This trend becomes evident when observing the reconstruction of the initial image, where the resemblance to the original remains pronounced.
Already with the first 10 eigenfaces (that capture 77% of the total variance) you can start to recognize Mila Kunis in the image.

Though the classes appear to occupy slightly different areas in the plotted PCA feature maps, there is quite a bit of overlap. This indicates that the classes may not be easily separable and that using principal components from these PCAs as input may not result in the most robust models.

1. # 2. Evaluation Metrics
## 2.0. Example: Accuracy


In [ ]:
from sklearn.metrics import accuracy_score

# 3. Classifiers


## 3.1 KNN Classifier¶


In [ ]:
class KNNCosineClassification():

  def __init__(self, knn=3):
    self.classifier = KNeighborsClassifier(n_neighbors=knn,metric="cosine")

  def fit(self, train_features, y_train):
    self.classifier.fit(train_features,y_train)

  def predict(self, x_val):
    return self.classifier.predict(x_val)

  def __call__(self,x_val):
    return self.predict(x_val)

In [ ]:
class KNNClassification():

  def __init__(self, knn=10):
    self.classifier = KNeighborsClassifier(n_neighbors=knn)

  def fit(self, train_features, y_train):
    self.classifier.fit(train_features,y_train)

  def predict(self, x_val):
    return self.classifier.predict(x_val)

  def __call__(self,x_val):
    return self.predict(x_val)

## 3.2 Decision tree classifier¶



In [ ]:
class DecisionTree():

    def __init__(self, random_state):
        self.classifier = DecisionTreeClassifier(random_state=random_state)

    def fit(self, train_features, y_train):
        self.classifier.fit(train_features, y_train)

    def predict(self, x_val):
        return self.classifier.predict(x_val)

    def __call__(self,x_val):
        return self.predict(x_val)

## 3.3 SVM classifier¶


In [ ]:
class SVM():
    def __init__(self):
        self.classifier = LinearSVC()

    def fit(self, train_features, y_train):
        self.classifier.fit(train_features, y_train)

    def predict(self, x_val):
        return self.classifier.predict(x_val)

    def __call__(self,x_val):
        return self.predict(x_val)

## 3.4. Baseline: VGG16 transfer learn model on original images
We first took the head of a VGG16 model and experimented with the number of hidden layers and number of neurons in the network. We did this to have a baseline of what a model could do without hand crafted features. The baseline achieved was 81.1%.

In [ ]:
# Imports

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, Model

In [ ]:
class OriginalImageClassificationModel:
    def __init__(self):
        base_model = tf.keras.applications.VGG16(weights = 'imagenet', include_top = False, input_shape = (100,100,3)) # have you considered using VGG19? https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG19
        for layer in base_model.layers:
          layer.trainable = False
        x = layers.Flatten()(base_model.output)
        x = layers.Dense(256, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(128, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        predictions = layers.Dense(3, activation = 'softmax')(x)
        self.model = Model(inputs = base_model.input, outputs = predictions)
        self.model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

    def fit(self, x_train, y_train, x_val, y_val, batch_size=64, epochs=40):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))
        return history

    def predict(self, X):
        return self.model.predict(X)

## 3.5 HOG-image based
In the literature we found that models using a HOG-image as input were able to perform well, so therefore we tested some models that could work.

### 3.5.1 Using a VGG16 pretrained model¶
The first model we tried was a VGG16 model as it was easy to implement. The problem with this is that VGG16 model requires RGB input so you need to stack 3 HOG-images before feeding it into the model.

We tested many different things such as changing the amount of linear layers, changing the number of neurons, batch size, and learning rate. In the end it got close to the baseline but could not surpass it.

In [ ]:
# Imports

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, Model, optimizers

In [ ]:
class HOGClassificationModel:
    def __init__(self):
        base_model = tf.keras.applications.VGG16(weights = 'imagenet', include_top = False, input_shape = (100,100,3))
        for layer in base_model.layers:
          layer.trainable = False
        x = layers.Flatten()(base_model.output)
        x = layers.Dense(4096, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(4096, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        predictions = layers.Dense(3, activation = 'softmax')(x)
        self.model = Model(inputs = base_model.input, outputs = predictions)
        self.model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

    def fit(self, x_train, y_train, x_val, y_val, batch_size=12, epochs=40):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))
        return history

    def predict(self, X):
        return self.model.predict(X)

### 3.4.2 Custom model
In a paper we found a model that performed really well on benchmark datasets. There for we copied the model and did some tests with it. We unfortunately were not able to replicate the good results on our dataset. In the paper they used 48x48 images so we tested it on both 48x48 and 100x100 and didn't notice a difference.

The problem with this model is that it was always able do well on the training set but it didn't transfer to the validation set. We suspect it was overfitting so we tried many different things to fix it, such as testing many scaled down models, augmenting the training data, regenerating the augmlented data every couple of epochs, and adding dropout layers (also between the conv2s layers).

After testing all these things we decided to give up on only HOG-image based deep learning classifiers

Source: Benradi, Hicham, Ahmed Chater, and Abdelali Lasfar. "A hybrid approach for face recognition using a convolutional neural network combined with feature extraction techniques." IAES Int J Artif Intell 12.2 (2023): 627.

In [ ]:
class HOGClassificationModel2:
    def __init__(self):
        self.model = models.Sequential()
        self.model.add(layers.Conv2D(6, 5, activation='relu', input_shape = (48,48,1)))
        self.model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(layers.Conv2D(16, 5, activation='relu'))
        self.model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(layers.Conv2D(64, 3, activation='relu'))
        self.model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(layers.Flatten())
        self.model.add(layers.Dense(256, activation='relu'))
        self.model.add(layers.Dropout(0.5))
        self.model.add(layers.Dense(3, activation = 'sigmoid'))
        self.model.compile(optimizer=optimizers.RMSprop(), loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])


    def fit(self, x_train, y_train, x_val, y_val, batch_size=32, epochs=40):
        history = self.model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(x_val, y_val))
        return history

    def predict(self, X):
        return self.model.predict(X)

## 3.5 Combined original images and HOG-image approach
In the end we decided to combine the previous models into one. So a VGG16 model that extracts features from the original image and the custom HOG-image model that extracts features from the HOG-image. These features are then combined into and given as input to 2 linear layers. After tweeking the network, we were finally able to beat the baseline model. Adding a HOG-image as input imrpoves the standard VGG model.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, losses, Model

!!! Disclaimer you have to parse a dict with the name of the input layer in the fit and predict function and depending on the amount of models that have been created these values change. When it is ran for the first time it are these values other wise they will be different.



In [ ]:
class ImageAndHogClassificationModel:
    def __init__(self):
        base_model = tf.keras.applications.VGG16(weights = 'imagenet', include_top = False, input_shape = (100,100,3)) # have you considered using VGG19? https://www.tensorflow.org/api_docs/python/tf/keras/applications/VGG19
        for layer in base_model.layers:
          layer.trainable = False

        model = models.Sequential()
        model.add(layers.Conv2D(6, 5, activation='relu', input_shape = (100,100,1)))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(16, 5, activation='relu'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Conv2D(64, 3, activation='relu'))
        model.add(layers.MaxPooling2D(pool_size=(2, 2)))
        model.add(layers.Flatten())

        y = layers.Flatten()(base_model.output)
        z = layers.concatenate([y, model.output])

        x = layers.Dense(1024, activation='relu')(y)
        x = layers.Dropout(0.5)(x)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        predictions = layers.Dense(3, activation = 'softmax')(x)
        self.model = Model(inputs = [base_model.input, model.input] , outputs = predictions)
        self.model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])

    def fit(self, x_train1, x_train2, y_train, x_val1, x_val2, y_val, batch_size=64, epochs=40):
        history = self.model.fit({'input_1': x_train1, "conv2d_input": x_train2}, y_train, batch_size=batch_size, epochs=epochs, validation_data=({'input_1': x_val1, "conv2d_input": x_val2}, y_val))
        return history

    def predict(self, X1, X2):
        return self.model.predict({'input_1': X1, "conv2d_input": X2})

# 4. Experiments

## 4.0. SIFT pipeline¶

First some experiments using SVM and KNN on the SIFT feature space:

In [ ]:
PARAMS = {
    "Octaves":4,
    "NumberOfFeatures": 100,
    "ConstrastThreshold": 0.01,
    "EdgeThreshold": 30,
    "Smoothing": 2.0,
}

classifier = SVM()
skf = StratifiedKFold(n_splits = 10)
mean_acc = []

for train_idx, test_idx in skf.split(new_train_X, new_train_y):
    x_train, y_train = new_train_X[train_idx,:,:,:], new_train_y[train_idx]
    x_val, y_val = new_train_X[test_idx,:,:,:], new_train_y[test_idx]
    trans = SIFT_FeatureExtractor(**PARAMS)
    all_keypoints, all_features = trans(x_train) #Note that these are LISTS with a length equal to the training set size
    trans.createKWords(all_features,numClusters)
    all_words = trans.getWords(all_features)
    classifier.fit(all_words, y_train)

    test_keypoints, test_features = trans(x_val)
    test_words = trans.getWords(test_features)

    y_pred = classifier(test_words)
    acc = accuracy_score(y_val, y_pred)
    mean_acc.append(acc)
    print(acc)
print("Accuracy from 10-fold cross validation on SIFT features in SVM classifier: ",np.mean(mean_acc))

In [ ]:
PARAMS = {
    "Octaves":4,
    "NumberOfFeatures": 100,
    "ConstrastThreshold": 0.01,
    "EdgeThreshold": 30,
    "Smoothing": 2.0,
}

classifier = KNNCosineClassification()
skf = StratifiedKFold(n_splits = 10)
mean_acc = []

for train_idx, test_idx in skf.split(new_train_X, new_train_y):
    x_train, y_train = new_train_X[train_idx,:,:,:], new_train_y[train_idx]
    x_val, y_val = new_train_X[test_idx,:,:,:], new_train_y[test_idx]
    trans = SIFT_FeatureExtractor(**PARAMS)
    all_keypoints, all_features = trans(x_train) #Note that these are LISTS with a length equal to the training set size
    trans.createKWords(all_features,numClusters)
    all_words = trans.getWords(all_features)
    classifier.fit(all_words, y_train)

    test_keypoints, test_features = trans(x_val)
    test_words = trans.getWords(test_features)

    y_pred = classifier(test_words)
    acc = accuracy_score(y_val, y_pred)
    mean_acc.append(acc)
    print(acc)
print("Accuracy from 10-fold cross validation on SIFT features in KNN classifier: ",np.mean(mean_acc))

In [ ]:
feature_extractor = IdentityFeatureExtractor() 
classifier = RandomClassificationModel()

# train the model on the features
classifier.fit(feature_extractor(train_X), train_y)

# model/final pipeline
model = lambda X: classifier(feature_extractor(X))

In [ ]:
# evaluate performance of the model on the training set
train_y_star = model(train_X)

"The performance on the training set is {:.2f}. This however, does not tell us much about the actual performance (generalisability).".format(
    accuracy_score(train_y, train_y_star))

In [ ]:
# predict the labels for the test set 
test_y_star = model(test_X)

## 4.1 HOG-image pipeline¶
With changing the augmented data every couple of epochs

Extract the HOG-images without stacking them



In [ ]:
hogfe = HOGFeatureExtractor()

hog_train_X = []

for i in range(len(train_X)):
  _, hog_image = hogfe.transform(train_X[i])
  hog_train_X.append([hog_image])
hog_train_X = np.array(hog_train_X).transpose(0,2,3,1)

A funciton to convert a whole list of images to HOG-images

In [ ]:
def hog_trans_list(train_list):
  hogfe = HOGFeatureExtractor()
  temp = []
  for i in range(len(train_list)):
    _, hog_image = hogfe.transform(train_list[i])
    temp.append([hog_image])
  return np.array(temp).transpose(0,2,3,1)


Extract the HOG-images without stacking them



In [ ]:
hogfe = HOGFeatureExtractor()

hog_train_X = []

for i in range(len(new_train_X)):
  _, hog_image = hogfe.transform(new_train_X[i])
  hog_train_X.append(np.repeat(hog_image[..., np.newaxis], 3, -1))
hog_train_X = np.array(hog_train_X)

A funciton to convert a whole list of images to HOG-images



In [ ]:
def hog_trans_list(train_list):
  hogfe = HOGFeatureExtractor()
  temp = []
  for i in range(len(train_list)):
    _, hog_image = hogfe.transform(train_list[i])
  hog_train_X.append(np.repeat(hog_image[..., np.newaxis], 3, -1))
  return np.array(temp)

Choose the model you want to



In [ ]:
model = HOGClassificationModel2()


Train the model with K-fold validation




In [ ]:
from sklearn.model_selection import StratifiedKFold


In [ ]:
skf = StratifiedKFold(n_splits = 4)
for train_idx, test_idx in skf.split(train_X, train_y):
    x_val, y_val = train_X[test_idx,:,:,:], train_y[test_idx]
    x_val, y_val = apply_augmentations(x_val, y_val)
    x_val = hog_trans_list(x_val)
    for _ in range(3):
      x_train, y_train = train_X[train_idx,:,:,:], train_y[train_idx]
      x_train, y_train = apply_augmentations(x_train, y_train)
      x_train = hog_trans_list(x_train)
      history = model.fit(x_train, y_train, x_val, y_val, batch_size=64, epochs=2)

Extract the HOG-images from the test set



In [ ]:
hogfe = HOGFeatureExtractor()

hog_test_X = []

for i in range(len(test_X)):
  _, hog_image = hogfe.transform(test_X[i])
  hog_test_X.append([hog_image])
hog_test_X = np.array(hog_test_X).transpose(0,2,3,1)

In [ ]:
hogfe = HOGFeatureExtractor()

hog_test_X = []

for i in range(len(test_X)):
  _, hog_image = hogfe.transform(test_X[i])
  hog_test_X.append(np.repeat(hog_image[..., np.newaxis], 3, -1))
hog_test_X = np.array(hog_test_X)

In [ ]:
test_y_star = model.predict(hog_test_X)
test_y_star = np.argmax(test_y_star, axis=1)

## 4.2 HOG-image and normal images pipeline¶
With changing the augmented data every couple of epochs

Use the HOG extraction functions from the previous pipeline



Choose your model

In [ ]:
model = ImageAndHogClassificationModel()


Train the model



In [ ]:
from sklearn.model_selection import StratifiedKFold


In [ ]:
skf = StratifiedKFold(n_splits = 4)
for train_idx, test_idx in skf.split(train_X, train_y):
    x_val1, y_val = train_X[test_idx,:,:,:], train_y[test_idx]
    x_val1, y_val = apply_augmentations(x_val1, y_val)
    x_val2 = hog_trans_list(x_val1)
    for _ in range(1):
      x_train1, y_train = train_X[train_idx,:,:,:], train_y[train_idx]
      x_train1, y_train = apply_augmentations(x_train1, y_train)
      x_train2 = hog_trans_list(x_train1)
      history = model.fit(x_train1, x_train2, y_train, x_val1, x_val2, y_val, batch_size=128, epochs=2)

Extract the HOG-images from the test set



In [ ]:
hogfe = HOGFeatureExtractor()

hog_test_X = []

for i in range(len(test_X)):
  _, hog_image = hogfe.transform(test_X[i])
  hog_test_X.append([hog_image])
hog_test_X = np.array(hog_test_X).transpose(0,2,3,1)

In [ ]:
hogfe = HOGFeatureExtractor()

hog_test_X = []

for i in range(len(test_X)):
  _, hog_image = hogfe.transform(test_X[i])
  hog_test_X.append(np.repeat(hog_image[..., np.newaxis], 3, -1))
hog_test_X = np.array(hog_test_X)

In [ ]:
test_y_star = model.predict(test_X, hog_test_X)
test_y_star = np.argmax(test_y_star, axis=1)

# 5. Publishing best results

In [ ]:
submission = test.copy().drop('img', axis = 1)
submission['class'] = test_y_star

submission

In [ ]:
submission.to_csv('submission.csv')

# 6. Discussion


The main lessons we have learned during this project are: data cleaning is a very important step and should be done at the beginning. 
While this was a relatively less arduous task given the size of the dataset, it adds layers of complications to the data loading mechanism.
Verifying and re-verifying that the pre-processors work as intended is key. 
Here we have only slightly tweaked the pre-processor, but that is not a robust solution since there is some degree of uncertainty linked to the order in which the faces are detected by the pre-processor.
Handcrafted features can be used to improve upon the standard CNN-based models.
However, handcrafted features require careful finetuning and consideration for what image aspects are considered discriminative for the task at hand and will generalise to unseen data.
There is a big difference between model performance on the test set versus using cross-validation on the training set

We believe the Nearest Neighbour model struggled with handcrafted features due to the limited ability to extrapolate to unseen data. While SVM worked better, it tended to overfit to the noise and the pictures of Sarah Nyland and Michael Cera, which was made clear by the poor performance on the validation and test sets. PCA may be more appropriate for data compression, tasks where interpretability is important, and situations with limited computational resources. SIFT provides an excellent framework for larger object detection but struggles with identifying finer details for a discriminative classifier. It is very good at separating Jesse from Mila, but not good at identifying the lookalikes.


If we had more time, we would implement pre-processing pipelines that have shown better results in similar cases. We also didn’t have the time to study the effect of each augmentation on the overall accuracy of models and we might have missed some useful augmentations. For our future classifiers, we would Investigate ensemble methods combining learned features with handcrafted features, as HOG and normal images together give better results. For Nearest Neighbour, not only looking at the closest labels but also looking at the total distance to the nearest Neighbour might improve the classifier. 

